## Querying Tutorial

When querying, you can specify several arguments using `fast_graphrag.QueryParam`:
- `only_context`: skips the answer generation step and simply returns the context retrieved from the graph that would have been passed to the LLM. This includes the nodes, edges and chunks selected, each associated with its relevance scored, as computed by page rank.
- `with_references`: it is possible to ask the LLM to add inline references to the chunks (and thus documents) used to generate an answer.

Furthermore it is possible to specify how many tokens to use for the context. The default values are:
- `entities_max_tokens = 4000`: number of context tokens reserved for the retrieved nodes (i.e., entities) and their descriptions.
- `relationships_max_tokens = 3000`: number of context tokens reserved for the retrieved edges (i.e., relations between entities) and their descriptions.
- `chunks_max_tokens = 9000`: number of context tokens reserved for the retrieved chunks.

#### Querying with `only_context`

In [ ]:
from fast_graphrag import GraphRAG, QueryParam

working_dir = "..."
DOMAIN = "..."
QUERIES = ["...", "..."]
ENTITY_TYPES = ["...", "..."]

grag = GraphRAG(
    working_dir=working_dir,
    domain=DOMAIN,
    example_queries="\n".join(QUERIES),
    entity_types=ENTITY_TYPES,
)

# Inserting some data
corpus = {
    "title1": "corpus1 ...",
    "title2": "corpus2 ...",
}
grag.insert(
    [f"{title}: {corpus}" for title, corpus in tuple(corpus.items())],
    metadata=[{"id": title} for title in tuple(corpus.keys())],
)

# Querying only context
query = "..."
answer = grag.query(query, QueryParam(only_context=True))

print(answer.response)  # ""
print(answer.context)  # {entities: [...], relations: [...], chunks: [...]}

#### Querying with `with_references`

In [ ]:
from fast_graphrag import GraphRAG, QueryParam

working_dir = "..."
DOMAIN = "..."
QUERIES = ["...", "..."]
ENTITY_TYPES = ["...", "..."]

grag = GraphRAG(
    working_dir=working_dir,
    domain=DOMAIN,
    example_queries="\n".join(QUERIES),
    entity_types=ENTITY_TYPES,
)

# Inserting some data
corpus = {
    "title1": "corpus1 ...",
    "title2": "corpus2 ...",
}
grag.insert(
    [f"{title}: {corpus}" for title, corpus in tuple(corpus.items())],
    metadata=[{"id": title} for title in tuple(corpus.keys())],
)

# Querying only context
query = "..."
answer = grag.query(query, QueryParam(with_references=True))

`answer` should now contain inline references in the form of `[<source_id>]`, where `source_id - 1` is the index of the corresponding chunk in `answer.context.chunks`. Each chunk comes with its own metadata, so you already extract all the information you want by parsing `answer.response`.
However, we provide a method that does that for you (and hopefully covers 99% of the use cases): `answer.format_references`.
`answer.format_references` takes a `format_fn` which allows you to specify what to write for each reference, in a tidy and easy way. Let's go through a couple of examples.
NOTE: if your use case is not covered by this, please feel free to create a feature request (or even a pull request) on GitHub, we'll be happy to have a look!

Let's consider the following query, answer and associated context (this is taken from the 2wikimultihopqa dataset):

```json
"query": "What is the place of birth of the performer of song Changed It?",
"answer": "The performer of the song \"Changed It\" is Nicki Minaj, who was born in Saint James, Port of Spain, Trinidad and Tobago, and was raised in Queens, New York City [1][4].",
"chunks": [
    [
        {
            "meta": {
                "id": "Changed It"
            },
            "content": "Changed It: \"Changed It\" is a song by Trinidadian-American rapper and singer Nicki Minaj and American rapper Lil Wayne..."
        },
        2.9243156630545855
    ],
    [
        {
            "meta": {
                "id": "Frank Sinatra"
            },
            "content": "Frank Sinatra: Francis Albert Sinatra  ( December 12, 1915 \u2013 May 14, 1998) was an American singer..."
        },
        0.3308500498533249
    ],
    [
        {
            "meta": {
                "id": "Andrew Allen (singer)"
            },
            "content": "Andrew Allen (singer): Andrew Allen( born 6 May 1981) is a Canadian- born singer- songwriter from Vernon, British Columbia..."
        },
        0.17779897525906563
    ],
    [
        {
            "meta": {
                "id": "Nicki Minaj"
            },
            "content": "Nicki Minaj: Onika Tanya Maraj-Petty (born December 8, 1982), known professionally as Nicki Minaj..."
        },
        1.392689185217023
    ],
    [
        {
            "meta": {
                "id": "Revolution (Jars of Clay song)"
            },
            "content": "Revolution (Jars of Clay song): \" Revolution\" is a song written and performed by Jars of Clay..."
        },
        0.13545062253251672
    ]
]
```
Please note a couple of things about the example above:
- Only retrieved chunks are shown (no entities or relationships even if those are provided to the LLM when generating the response);
- Chunk content is truncated to make it easy to visualize the list;
- Chunk "Nicki Minaj" has been manually moved from position #2 to position #4, and the corresponding reference in the answer has been changed from "[2]" to "[4]", to enphasize that `format_references` will automatically order the references for you;
- as mentioned early, reference indices start from `1`.

Let's now use `format_references`. `format_references` takes three parameters:
- `document_index`: a automatically generated increasing index associated to each referenced document;
- `chunk_indices`: a list of chunk indices associated with the document;
- `metadata`: the metadata dictionary associated to each document.
`document`s are created by aggregating all the chunks with the same metadata dictionary. In our case, each chunk is its own document since they all have different metadatas.

##### Example #1

In [ ]:
fresponse, fcontext = answer.format_references(
    lambda doc_index, chunk_indices, metadata: f"[{doc_index}.{'-'.join(map(str, chunk_indices))}]({metadata['id']})"
)

This output the following:

```json
{
    "fresponse": "The performer of the song \"Changed It\" is Nicki Minaj, who was born in Saint James, Port of Spain, Trinidad and Tobago, and was raised in Queens, New York City [1.1](Changed It)[2.1](Nicki Minaj).",
    "fcontext": {
            "1": {
                "meta": {
                    "id": "Changed It"
                },
                "chunks": {
                    "1": ["Changed It: \"Changed It\" is a song by Trinidadian-American...", 3258430810805445470]
                }
            },
            "2": {
                "meta": {
                    "id": "Nicki Minaj"
                },
                "chunks": {
                    "1": ["Nicki Minaj: Onika Tanya Maraj-Petty (born December 8, 1982)...", 18375001403800070367]
                }
            }
        }
}
```

If, instead, both "Changed It" and "Nicki Minaj" were from the same document with metadata `{"url": "wikipedia.com/changed-it-nicki-minaj"}` we could get:

In [ ]:
fresponse, fcontext = answer.format_references(
    lambda doc_index, chunk_indices, metadata: f"[{doc_index}]({metadata['url']})"
)

```json
{
    "fresponse": "The performer of the song \"Changed It\" is Nicki Minaj, who was born in Saint James, Port of Spain, Trinidad and Tobago, and was raised in Queens, New York City [1](wikipedia.com/changed-it-nicki-minaj).",
    "fcontext": {
            "1": {
                "meta": {
                    "url": "wikipedia.com/changed-it-nicki-minaj"
                },
                "chunks": {
                    "1": ["Changed It: \"Changed It\" is a song by Trinidadian-American...", 3258430810805445470],
                    "2": ["Nicki Minaj: Onika Tanya Maraj-Petty (born December 8, 1982)...", 18375001403800070367]
                }
            }
        }
}
```

Note that the formatting function has been arbitrarely changed to only output the document index and ignore the chunk_indices.